<a href="https://colab.research.google.com/github/jerry2eden/hamoye_excersie/blob/master/Samplepipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install kfp --upgrade --user

In [2]:
!which dsl-compile

In [5]:

import numpy as np
import pandas as  pd
import os
import matplotlib.pyplot as plt

# data encoding
from sklearn.preprocessing import LabelEncoder

# splitting the data
from sklearn.model_selection import train_test_split

# Standardization - feature scaling
from sklearn.preprocessing import StandardScaler

# Deep learning Libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

# evaluating the mode
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
out_dir = '/content/sample_data/DATA'

In [7]:

def train(data_path, model_file):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    from tensorflow.python import keras
    # splitting the data
    from sklearn.model_selection import train_test_split
    # Standardization - feature scaling
    from sklearn.preprocessing import StandardScaler
    # data encoding
    from sklearn.preprocessing import LabelEncoder
    
    # Download the dataset and split into training and test data. 
    data = pd.read_csv("https://raw.githubusercontent.com/AdeloreSimiloluwa/Artificial-Neural-Network/master/data/Churn_Modelling.csv")
    
    #preprocessing
    X = data.iloc[:, 3:-1]
    y = data.iloc[:,-1:]
    
    # encoding country
    encoder_X_1= LabelEncoder()
    X.iloc[:,1] = encoder_X_1.fit_transform(X.iloc[:,1])
    
    # encoding gender
    encoder_X_2= LabelEncoder()
    X.iloc[:,2] = encoder_X_2.fit_transform(X.iloc[:,2])
    
    # we would also use the dummy variable because they are norminal variables
    dummy = pd.get_dummies(X["Geography"], prefix = ['Geography'],drop_first=True)
    X=pd.concat([X,dummy], axis = 1)
    X.head()
    X=X.drop(columns = ['Geography'], axis = 1)
    
    # split the data
    X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.2, random_state = 0)
    # Normalize the data.
    sc =StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    y_train = y_train.values
    y_test = y_test.values
    
    # Define the model using Keras.
    classifier = keras.models.Sequential()
    classifier.add(keras.layers.Dense(units =6, activation='relu', input_dim=11))
    classifier.add(keras.layers.Dense(units =6, activation='relu'))
    classifier.add(keras.layers.Dense(units =1, activation='sigmoid'))

    classifier.summary()

    classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics =['accuracy'])

    # Run a training job with specified number of epochs
    classifier.fit(X_train, y_train, batch_size=10 , epochs=50 )

    #Evaluate the model and print the results
    test_loss, test_acc = classifier.evaluate(X_test,  y_test, verbose=0)
    print('Test accuracy:', test_acc)
    #Save the model to the designated 
    classifier.save(f'{data_path}/{model_file}')

    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/test_data', 'wb') as f:
        pickle.dump((X_test,  y_test), f)


In [ ]:
classifier = train(out_dir, "model")

In [9]:
def predict(data_path, model_file):
    
    import pickle
    import tensorflow as tf
    from tensorflow import keras
    import numpy as np
    
    # Load the saved Keras model
    classifier = keras.models.load_model(f'{data_path}/{model_file}')

    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test,  y_test = test_data

    # make predictions.
    y_pred = classifier.predict(X_test)

    # create a threshold
    y_pred=(y_pred>0.5)
    
    with open(f'{data_path}/result.txt', 'w') as result:
        result.write(" Prediction: {}, Actual: {} ".format(y_pred,y_test.astype(np.bool)))
    
    print('Prediction has be saved successfully!')

In [ ]:
predict(out_dir, "model")

In [11]:
# Create train and predict lightweight components.
train_op = comp.func_to_container_op(train , base_image = "tensorflow/tensorflow:latest-gpu-py3")
predict_op = comp.func_to_container_op(predict , base_image = "tensorflow/tensorflow:latest-gpu-py3")

In [12]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

Failed to load kube config.
